In [1]:
from aicsimageio import AICSImage
import os

import pandas as pd
import numpy as np

from napari import Viewer

from tifffile import imread

In [2]:
file = "/home/tibuch/Data/hochtobi/acq_1.nd"
nuc_cyto_seg_dir = "/home/tibuch/Gitrepos/gchao_RPL-Myc/runs/example/01_nuclei-cyto-segmentation/"
spot_dir = "/home/tibuch/Gitrepos/gchao_RPL-Myc/runs/example/02_spot-detection"

In [3]:
name, ext = os.path.splitext(os.path.basename(file))

In [4]:
img = AICSImage(file)

In [5]:
raw_01 = img.data[0, 1]
raw_02 = img.data[0, 2]

In [6]:
nuc_seg = imread(os.path.join(nuc_cyto_seg_dir, name + '_NUC-SEG-3D.tif'))
cyto_seg = imread(os.path.join(nuc_cyto_seg_dir, name + '_CYTO-SEG-2D.tif'))

In [20]:
spots_01 = pd.read_csv(os.path.join(spot_dir, name + "_SPOTS_C01.csv"))
spots_02 = pd.read_csv(os.path.join(spot_dir, name + "_SPOTS_C02.csv"))

In [8]:
sx = img.ome_metadata.dict()['images'][0]['pixels']['physical_size_x']
sy = img.ome_metadata.dict()['images'][0]['pixels']['physical_size_y']
sz = img.ome_metadata.dict()['images'][0]['pixels']['physical_size_z']
spacing = tuple([sz, sy, sx])

In [9]:
spacing = tuple([0.2, 0.103, 0.103])

In [26]:
viewer = Viewer()

In [27]:
viewer.add_image(raw_01, scale=spacing, name="C02", contrast_limits=np.quantile(raw_01, [0.6, 0.999]), colormap="magenta")
viewer.add_image(raw_02, scale=spacing, name="C03", contrast_limits=np.quantile(raw_02, [0.6, 0.999]), colormap="green", blending='additive')
viewer.add_labels(cyto_seg, scale=spacing[1:], name="cyto")
viewer.layers["cyto"].contour = 1
viewer.add_labels(nuc_seg, scale=spacing, name="nuclei", opacity=0.5)

<Labels layer 'nuclei' at 0x7f421c664550>

In [28]:
viewer.add_points(spots_01.query("in_nuclei & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="nuc-spots-01", symbol='o')

<Points layer 'nuc-spots-01' at 0x7f4230276d90>

In [29]:
viewer.add_points(spots_01.query("in_cyto & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="cyto-spots-01", symbol='square')

<Points layer 'cyto-spots-01' at 0x7f42512b9850>

In [30]:
viewer.add_points(spots_02.query("in_nuclei & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="nuc-spots-02", symbol='o')

<Points layer 'nuc-spots-02' at 0x7f4251734880>

In [31]:
viewer.add_points(spots_02.query("in_cyto & symmetric")[["centroid_z", "centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="cyto-spots-02", symbol='square')

<Points layer 'cyto-spots-02' at 0x7f42515c83a0>

In [ ]:
#screenshot = viewer.screenshot(scale=5, path='./screenshot_slice.png')

In [ ]:
viewer.add_image(np.max(raw_01, 0), scale=spacing[1:], name="C01", contrast_limits=np.quantile(np.max(raw_01, 0), [0.6, 0.999]), colormap="magenta")
viewer.add_image(np.max(raw_02, 0), scale=spacing[1:], name="C02", contrast_limits=np.quantile(np.max(raw_02, 0), [0.6, 0.999]), colormap="green", blending='additive')
viewer.add_labels(cyto_seg, scale=spacing[1:], name="cyto")
viewer.layers["cyto"].contour = 1
viewer.add_labels(np.max(nuc_seg, 0), scale=spacing[1:], name="nuclei", opacity=0.5)

In [ ]:
viewer.add_points(spots_01.query("in_nuclei & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="nuc-spots-01", symbol='o')

In [ ]:
viewer.add_points(spots_01.query("in_cyto & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='magenta',
                   face_color='#00000000', name="cyto-spots-01", symbol='square')

In [ ]:
viewer.add_points(spots_02.query("in_nuclei & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="nuc-spots-02", symbol='o')

In [ ]:
viewer.add_points(spots_02.query("in_cyto & symmetric")[["centroid_y", "centroid_x"]].to_numpy(), size=1, edge_color='green',
                   face_color='#00000000', name="cyto-spots-02", symbol='square')

In [ ]:
#screenshot = viewer.screenshot(scale=5, path='./screenshot_projection.png')